# Knowledge Gradient

In [1]:
import GPy
import copy
import numpy as np
import scipy as spy
import matplotlib.pyplot as plt

### Synth Example
We want to use knowledge gradient to see if we can efficiently sample and learn the following noisy function. 
$$f(x) = 0.50 \times x + \mathcal{N}(0, 1)$$
For the sake of this example, we assume that $x$ is constrained between $[0,1]$

In [2]:
# Define f(x)
def f(x):
    return 0.50*x + np.random.normal(0.0, 1.0)

# Establish Bounds on x
x_bounds = [0.0, 1.0]

## Surrogate 
For this example, given that we have a uniform noise level, we will contend with using a vanilla GP with RBF and White kernel.

In [3]:
kernel = GPy.kern.RBF(input_dim=1) + GPy.kern.White(input_dim=1)

To start, we will uniformly sample the solution space and fit the GP on that data. This enable us to get a lay of the land.

In [4]:
initial_samples = 5

# Generate random samples
X = np.random.random_sample(initial_samples)
Y = np.zeros(initial_samples)
for idx, i in enumerate(X):
    Y[idx] = f(i)

# Normalize Data
Y = (Y - np.min(Y))/(np.max(Y)-np.min(Y))

Using these X,Y, train a GP

In [5]:
X = X.reshape(-1,1)
Y = Y.reshape(-1,1)
gp = GPy.models.GPRegression(X, Y, kernel)
gp.optimize(messages=False)
gp.optimize_restarts(num_restarts = 2)

Optimization restart 1/2, f = 3.316435415830541
Optimization restart 2/2, f = 3.316435415851991


## Acquisition Function

We will be implementing knowledge gradient acquisition function.

In [6]:
R = 10   # Number of restarts 
J = 10 # Number of replications
T = 100  # Number of steps 
a = 4    # Step size 


# Find a x with largest KG (Algorithm 3)
x_r = np.zeros(R)
KG_r = np.zeros(R)
for r in range(0, R):
    x_r[r] = np.random.random_sample(1) # Generate random sample from solution space, in this case [0,1]
    
    x_t = copy.deepcopy(x_r[r])
    for t in range(0, T):
        # Make a copy of main GP for local use
        gp_acquision = copy.deepcopy(gp)
        # Estimate Stocastic Gradient (Algorithm 4)
        G = 0.0 # Gradient
        for j in range(0, J):
            
            Z = np.random.normal(0.0, 1.0)
            # Estimate the mean, var from gp trained on previous samples for the proposed x 
            mu, var = gp_acquision.predict(x_t.reshape(-1, 1))
            # Estimates the value of intial staring point x^r_0
            y_np1 = mu + var * Z # y_np1 = y_{n+1}
            
            # Update gp with new x,y
            temp_X = gp_acquision.X
            temp_Y = gp_acquision.Y
            
            X_a = np.vstack((temp_X, x_t))
            Y_a = np.vstack((temp_Y, y_np1))
            # Update the data but retain the model
            gp_acquision.set_XY(X_a, Y_a)
            
            # Using the updated model, find the x that maximizes mean
            def mu_np1(x): # Estimate of mu_{n+1}, refer to Algorithm 4 for specifics
                mu, _ = gp_acquision.predict(x.reshape(-1, 1))
                return mu
            x_star, mu_star, _ = spy.optimize.fmin_l_bfgs_b(mu_np1, x0=x_t, approx_grad=1, bounds=[(0.0,1.0)])
            
            # Compute gradient of mu_{n+1} wrt x^r keeping x_star constant
            h = 0.01 # Preturbation to estimate gradient
            x_h = x_t + h
            
            # Update GP back to n samples
            gp_acquision.set_XY(temp_X, temp_Y)
            mu_h, var_h = gp_acquision.predict(x_h.reshape(-1, 1))
            y_h = mu_h + var_h * Z # y_np1 = y_{n+1}
            
            X_a = np.vstack((temp_X, x_h))
            Y_a = np.vstack((temp_Y, y_h))
            # Update the data but retain the model
            gp_acquision.set_XY(X_a, Y_a)
            mu_h, _ = gp_acquision.predict(x_star.reshape(-1, 1))
            
            G += (mu_h - mu_star)/h
        
        # Gradient KG
        delta_KG = G/J
        
        # Update the stepsize 
        at = (a)/(a + t)
        
        # Update with gradient ascent
        x_t = x_t + at * delta_KG
    
    x_r[r] = copy.deepcopy(x_t)
    # Now we have found out x. Lets estimate the knowledge gain for choosing that x (Algorithm 2)
    
    def mu_n(x): # Estimate of mu_n, refer to Algorithm 2 for specifics
        mu, _ = gp.predict(x.reshape(-1, 1))
        return mu
    
    # Best performing x in n samples
    _, mu_star_n, _ = spy.optimize.fmin_l_bfgs_b(mu_np1, x0=x_t, approx_grad=1, bounds=[(0.0,1.0)])
    
    mu, var = gp.predict(x_t.reshape(-1,1))
    gp_kg = copy.deepcopy(gp)
    delta_kg = 0.0
    for j in range(0, J):
        
        # Predicited performance of this sampling at this x_t
        y_np1 = np.random.normal(mu, var)
        
        # If sampled at this point, how the overall estimate change
        X_kg = np.vstack((temp_X, x_t))
        Y_kg = np.vstack((temp_Y, y_np1))
        # Update the data but retain the model
        gp_kg.set_XY(X_kg, Y_kg)
        
        def mu_np1_kg(x): # Estimate of mu_n, refer to Algorithm 2 for specifics
            mu, _ = gp_kg.predict(x.reshape(-1, 1))
            return mu
    
        # Best performing x in n samples
        _, mu_star_np1, _ = spy.optimize.fmin_l_bfgs_b(mu_np1_kg, x0=x_t, approx_grad=1, bounds=[(0.0,1.0)])
        
        delta_kg += mu_star_np1 - mu_star_n
    
    KG_r[r] = delta_kg/J

In [10]:
x_r[np.argmax(KG_r)]

0.530083280091131